# Reinforcement Learning

## Row Gridword Resolution

In [1]:
class GridWorldRow():
    def __init__(self, size=5):
        self.size = size
        self.nS = size # state number
        self.nA = 2 # action number (right, left)
        self.MAX_X = size-1 # max state, we start to index from 0
        P = {} # set up the environment {p:{a:[(p(s'|s,a), s', r(s,a,s'), done)]}

        for s in range(self.nS):

            dynamic_s = {}

            for a in range(self.nA):

                s_prime_list = []
                p = 1 if s != 0 and s != self.nS-1 else 0

                if a == 0:
                    s_prime = max(0, s-1)
                else:
                    s_prime = min(self.MAX_X, s+1)
                
                if s_prime == 0:
                    reward = -100
                    done = True
                elif s_prime == self.MAX_X:
                    reward = 10
                    done = True
                else:
                    reward = 0
                    done = False
                
                s_prime_list.append((p, s_prime, reward, done))
                dynamic_s.update({a:s_prime_list})

            P.update({s: dynamic_s})

        self.P = P # assign the environment

    def __str__(self):
        return f"{self.P}"

## Basic Algorithms

In [2]:
import numpy as np

In [3]:
env = GridWorldRow(size = 100)
print(env)

{0: {0: [(0, 0, -100, True)], 1: [(0, 1, 0, False)]}, 1: {0: [(1, 0, -100, True)], 1: [(1, 2, 0, False)]}, 2: {0: [(1, 1, 0, False)], 1: [(1, 3, 0, False)]}, 3: {0: [(1, 2, 0, False)], 1: [(1, 4, 0, False)]}, 4: {0: [(1, 3, 0, False)], 1: [(1, 5, 0, False)]}, 5: {0: [(1, 4, 0, False)], 1: [(1, 6, 0, False)]}, 6: {0: [(1, 5, 0, False)], 1: [(1, 7, 0, False)]}, 7: {0: [(1, 6, 0, False)], 1: [(1, 8, 0, False)]}, 8: {0: [(1, 7, 0, False)], 1: [(1, 9, 0, False)]}, 9: {0: [(1, 8, 0, False)], 1: [(1, 10, 0, False)]}, 10: {0: [(1, 9, 0, False)], 1: [(1, 11, 0, False)]}, 11: {0: [(1, 10, 0, False)], 1: [(1, 12, 0, False)]}, 12: {0: [(1, 11, 0, False)], 1: [(1, 13, 0, False)]}, 13: {0: [(1, 12, 0, False)], 1: [(1, 14, 0, False)]}, 14: {0: [(1, 13, 0, False)], 1: [(1, 15, 0, False)]}, 15: {0: [(1, 14, 0, False)], 1: [(1, 16, 0, False)]}, 16: {0: [(1, 15, 0, False)], 1: [(1, 17, 0, False)]}, 17: {0: [(1, 16, 0, False)], 1: [(1, 18, 0, False)]}, 18: {0: [(1, 17, 0, False)], 1: [(1, 19, 0, False)]},

### Policy Evalutation

In [4]:
pi = np.ones([env.nS, env.nA]) * 0.5
V = np.zeros([env.nS, 1])

gamma = 0.99 #replacement factor the return
theta = 1e-5 #similarity threshold to stop update

In [5]:
def compute_q_value_for_s_a(env, V, s, a, gamma):
    q = 0
    for (p_sPrime, sPrime, r_ss_a, done) in env.P[s][a]:
        q += p_sPrime * (r_ss_a + gamma * V[sPrime])
    return q

In [6]:
i = 0
while True:
    i += 1
    delta = 0

    for s in range(env.nS):
        V_new = 0

        for a in range(env.nA):
            prob_a = pi[s][a]
            q_s_a = compute_q_value_for_s_a(env, V, s, a, gamma)
            V_new += prob_a * q_s_a
        
        delta = max(delta, np.abs(V_new - V[s]))
        V[s] = V_new

    if delta < theta:
        print(f"Done after {i} iterations")
        break

Done after 417 iterations


In [7]:
V

array([[ 0.00000000e+00],
       [-8.76372108e+01],
       [-7.60347709e+01],
       [-6.59683891e+01],
       [-5.72347050e+01],
       [-4.96572812e+01],
       [-4.30830390e+01],
       [-3.73791661e+01],
       [-3.24304332e+01],
       [-2.81368667e+01],
       [-2.44117281e+01],
       [-2.11797627e+01],
       [-1.83756787e+01],
       [-1.59428283e+01],
       [-1.38320634e+01],
       [-1.20007426e+01],
       [-1.04118698e+01],
       [-9.03334685e+00],
       [-7.83732495e+00],
       [-6.79964225e+00],
       [-5.89933560e+00],
       [-5.11821711e+00],
       [-4.44050667e+00],
       [-3.85251321e+00],
       [-3.34235807e+00],
       [-2.89973509e+00],
       [-2.51570233e+00],
       [-2.18250147e+00],
       [-1.89340109e+00],
       [-1.64256065e+00],
       [-1.42491251e+00],
       [-1.23605958e+00],
       [-1.07218644e+00],
       [-9.29982329e-01],
       [-8.06574206e-01],
       [-6.99468738e-01],
       [-6.06501923e-01],
       [-5.25795379e-01],
       [-4.5

### Policy Improvement

In [8]:
for s in range(env.nS):
    q_s = np.zeros([env.nA, 1])

    for a in range(env.nA):
        q_s[a] = compute_q_value_for_s_a(env, V, s, a, gamma)

    best_a = np.argmax(q_s)
    pi[s] = np.eye(env.nA)[best_a]

In [9]:
pi

array([[1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.

### Policy Iteration

In [10]:
def evaluate_policy(env, pi, V, gamma, theta):
    V_updated = np.copy(V)
    improved = True

    while True:
        delta = 0

        for s in range(env.nS):
            V_new = 0

            for a in range(env.nA):
                prob_a = pi[s][a]
                q_s_a = compute_q_value_for_s_a(env, V, s, a, gamma)
                V_new += prob_a * q_s_a
            
            delta = max(delta, np.abs(V_new - V_updated[s]))
            V_updated[s] = V_new

        if delta < theta:
            break
    
    if (np.array_equal(V, V_updated)):
        improved = False

    return V_updated, improved

def improve_policy(env, pi, V, gamma):
    for s in range(env.nS):
        q_s = np.zeros([env.nA, 1])

        for a in range(env.nA):
             q_s[a] = compute_q_value_for_s_a(env, V, s, a, gamma)

        best_a = np.argmax(q_s)
        pi[s] = np.eye(env.nA)[best_a]

    return pi

In [11]:
def policy_iteration(env, pi, V, gamma, theta):
    i = 0
    while True:
        i += 1
        V, improved = evaluate_policy(env, pi, V, gamma, theta)
        pi = improve_policy(env, pi, V, gamma)

        if improved == False:
            print(f"Done after {i} step")
            break
    return pi, V

In [12]:
pi = np.ones([env.nS, env.nA]) * 0.5
V = np.zeros([env.nS, 1])

gamma = 0.99 #replacement factor the return
theta = 1e-5 #similarity threshold to stop update

In [13]:
pi, V = policy_iteration(env, pi, V, gamma, theta)
pi

Done after 100 step


array([[1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.

### Modified Policy Iteration

In [ ]:
def evaluate_policy(env, pi, V, gamma, k):
    V_updated = np.copy(V)
    improved = True

    for _ in range(k):
        delta = 0

        for s in range(env.nS):
            V_new = 0

            for a in range(env.nA):
                prob_a = pi[s][a]
                q_s_a = compute_q_value_for_s_a(env, V, s, a, gamma)
                V_new += prob_a * q_s_a
            
            delta = max(delta, np.abs(V_new - V_updated[s]))
            V_updated[s] = V_new
    
    if (np.array_equal(V, V_updated)):
        improved = False

    return V_updated, improved

def improve_policy(env, pi, V, gamma):
    for s in range(env.nS):
        q_s = np.zeros([env.nA, 1])

        for a in range(env.nA):
             q_s[a] = compute_q_value_for_s_a(env, V, s, a, gamma)

        best_a = np.argmax(q_s)
        pi[s] = np.eye(env.nA)[best_a]

    return pi